# FASTQ to FASTA converter
**Barbera Thomas 845538**

## Richiesta
Si richiede di scrivere un convertitore da FASTQ a FASTA che prenda in input un file di reads in formato FASTQ e produca in formato FASTA i soli reads che hanno le seguenti caratteristiche:
* non sono più corti di una soglia L1 e non sono più lunghi di una soglia L2
* la qualità minima delle basi supera una soglia Q1
* contengono una sottoregione con qualità minima Q2 (maggiore di Q1) che è lunga almeno P% della lunghezza del read

Viene richiesto l'uso di Biopython per leggere i reads dal file FASTQ in input e per stampare (in standard output o su file) i reads in formato FASTA.

## Input
Devono essere indicati il nome del file FASTQ di input e il nome del file FASTA di output.
È inoltre necessario fornire le seguenti variabili:
L1, L2 (> L1), Q1, Q2 (> Q1) e P.

## Output
Il programma scriverà le reads elaborate nel file di output indicato.
Per ognuno dei reads in output, l'header FASTA deve contenere le seguenti informazioni:
* identificatore
* lunghezza
* qualità minima delle basi
* start ed end della sottoregione con qualità minima Q2
* qualità media della sottoregione con qualità minima Q2

---

In [ ]:
# Input del programma:
input_file_name = './input.fq'
output_file_name = './output.fa'
L1 = 10
L2 = 40
Q1 = 24
Q2 = 60
P = 0.32  # è una percentuale - deve essere compreso tra 0 e 1

Per poter eseguire questo Notebook è necessario aver installato il pacchetto `biopython`.

In [ ]:
# Importazione dei package necessari:
import Bio
from Bio import SeqIO
from Bio import SeqRecord
from Bio.SeqRecord import SeqRecord
import math

La seguente lista conterrà i record che rispettano i vincoli. Verrà riempita durante l'analisi del file di input e servirà a scrivere l'output in formato FASTA.

In [ ]:
fasta_records = []

### Definizione delle funzioni necessarie

In [ ]:
def check_length(seq):
    # Si verifica che la lunghezza del read sia compresa tra L1 e L2.
    return len(seq)>=L1 and len(seq)<=L2

In [ ]:
def check_quality(quality_list):
    # Si verifica che la qualità minima delle basi superi la soglia Q1.
    return min(quality_list) > Q1

In [ ]:
def check_subregion(quality_list):
    # Si controlla l'esistenza di una sottoregione con qualità minima Q2 lunga
    # almeno P% della lunghezza del read. La funzione ritorna una tupla con la
    # posizione di inizio e di fine della più lunga regione che rispetta
    # tale vincolo (se esistente). Se non viene trovata nessuna regione è restituita
    # la tupla (-1, -1).
    bool_list = [q >= Q2 for q in quality_list]
    # Ora si deve trovare la più lunga regione di 'True' consecutivi nella lista.
    start = -1
    end = -1
    i = 0
    j = 0
    
    while i < len(bool_list):
        if not bool_list[i]:
            i += 1
            continue
        j = i
        while j < len(bool_list) and bool_list[j]:
            j += 1
        if j-i > end-start:
            start = i
            end = j
        i = j
    # Alla fine di questo ciclo, 'start' e 'end' conterranno la posizione di inizio
    # (inclusa) e quella di fine (esclusa) della regione più lunga con qualità minima
    # Q2. È solo necessario verificare che la sua lunghezza sia almeno P% della
    # lunghezza del read.
    required_length = math.ceil(len(quality_list) * P)
    if end-start >= required_length:
        # La sottoregione trovata è accettata.
        # Vengono restituite le posizioni di inizio e fine incluse (zero based).
        return (start, end-1)
    else:
        # Non è stata trovata nessuna regione valida.
        return (-1, -1)

In [ ]:
def list_avg(list):
    return round(sum(list) / len(list), 2)

La seguente funzione analizza un record FASTQ e controlla se vengono rispettati tutti i vincoli imposti. In tal caso, procede ad inserire il record nella lista `fasta_records`, in modo da poter poi essere scritto nel file di output.

In [ ]:
def check_record(record):
    quality_seq = record.letter_annotations['phred_quality']
    # Vengono ora verificate le diverse condizioni tramite le funzioni apposite.
    valid_length = check_length(str(record.seq))
    valid_quality = check_quality(quality_seq)
    subregion = check_subregion(quality_seq)
    
    if valid_length and valid_quality and subregion[0]!=-1 and subregion[1]!=-1:
        # Le condizioni sono rispettate, questo record verrà aggiunto al
        # file FASTA di output. Effettuiamo quindi l'append alla lista.
        record_to_add = SeqRecord(record.seq)
        record_to_add.id = record.id
        record_to_add.description = (
            "length: " + str(len(record.seq)) + ", " +
            "min_quality: " + str(min(quality_seq)) + ", " +
            "subregion: " + str(subregion[0]+1) + "-" + str(subregion[1]+1) + ", " +
            "subregion_avg_quality: " + str(list_avg(quality_seq[subregion[0]:subregion[1]+1]))
        )
        
        fasta_records.append(record_to_add)

### Lettura dell'input, analisi e scrittura del file di output

Prima di iniziare ad analizzare i record del file si verifica che le veriabili passate in input siano nel formato corretto e che rispettino i vincoli indicati nelle specifiche.

In [ ]:
if isinstance(L1, int) and isinstance(L2, int) and isinstance(Q1, (int, float)) and isinstance(Q2, (int, float)) and isinstance(P, (int, float)) and L2>L1 and Q2>Q1 and P>=0 and P<=1:
    # I parametri in input sono corretti, è possibile parsare il file di input.
    fastq_records = list(SeqIO.parse(input_file_name, 'fastq'))
    for r in fastq_records:
        check_record(r)
    # Ora che è stata riempita la lista dei record è possibile scrivere il file.
    SeqIO.write(fasta_records, output_file_name, 'fasta')
    print("È stato creato il file '" + output_file_name + "'.")
else:
    # Sono stati forniti parametri in un formato scorretto. La computazione termina.
    print("Parametri in input non validi.")